# Noise (and signal) correlations

This notebook analyzes the noise and signal correlation data calculated by the `scripts/pairwise_correlations.py` script.

In [ ]:
import <- function(pkg) { library(pkg, warn.conflicts=F, quietly=T, character.only=T) }
import("repr")
import("stringr")
import("tidyr")
import("dplyr")
import("ggplot2")
import("lme4")
import("emmeans")
import("xtable")

In [ ]:
options(repr.matrix.max.cols=15, repr.matrix.max.rows=20)
my.theme <- theme(legend.text=element_text(size=6),
                  legend.title=element_text(size=6),
                  plot.title = element_text(size=7, hjust=0.5),
                  axis.line=element_line(linewidth=0.25),
                  axis.ticks=element_line(linewidth=0.25),
                  axis.ticks.length=unit(0.05, "cm"),
                  axis.title=element_text(size=7),
                  axis.text=element_text(size=6),
                  strip.placement="outside",
                  strip.text=element_text(size=7),
                  strip.background=element_blank())
no.legend <- theme(legend.position="none")
update_geom_defaults("point", list(fill="white", shape=21, size=0.8))
update_geom_defaults("line", list(linewidth=0.4))


## Loading data and first steps of analysis

In [ ]:
# Metadata
birds <- (
    data.table::fread("../inputs/bird_metadata.csv")
    |> filter(behavior == "no")
)
sites <- data.table::fread("../inputs/recording_metadata.csv")
all_sites <- (
   sites
   |> mutate(bird=str_match(site, "[:alnum:]+")[,1])
   |> inner_join(birds, by="bird")
   |> filter(area %in% c("deep", "intermediate", "superficial"), stim_amplitude == "okay")
   |> mutate(area=forcats::fct_recode(area, "L1/CM"="superficial", "L2a/L2b"="intermediate", "L3/NCM"="deep") |> forcats::fct_relevel(c("L2a/L2b", "L1/CM", "L3/NCM")),
             group=factor(group, levels=c("CR", "PR")))
)

In [ ]:
# Need to have run `python scripts/unit_waveforms.py -o build inputs/all_units.txt`
unit_spike_features <- (
    data.table::fread("../build/mean_spike_features.csv") %>%
    mutate(spike=factor(spike, levels=c("wide", "narrow"), exclude="")) %>%
    filter(!is.na(spike))
)
# Need to have run `single-unit-analysis` notebook to identify responsive units
responsive_units <- data.table::fread("../inputs/responsive_units.txt", header=F, col.names=c("unit"))
    
# Need to have run `scripts/extract_channel.py inputs/all_units.tbl > build/unit_channels.csv"
units <- (
    data.table::fread("../build/unit_channels.csv")
    |> semi_join(responsive_units, by="unit")
    |> inner_join(unit_spike_features |> select(unit, spike), by="unit")
)

In [ ]:
# Pairwise correlations. Need to have run `batch/pairwise_correlations.sh < inputs/recording_metadata.csv`u
header <- data.table::fread(cmd='find ../build/ -name "*_correlations.csv" | head -n1 | xargs head -n1', header=T)
unit_correlations <- tibble(data.table::fread(cmd='find ../build/ -name "*_correlations.csv" | xargs tail -q -n+2', header=F))
names(unit_correlations) <- names(header)

In [ ]:
ucorr <- (
    unit_correlations
    # drop all comparisons where signal or noise correlation can't be calculated (typically because responses are too weak)
    |> filter(!is.na(evoked_noise), !is.na(signal))
    # look up channel and spike type. This will also remove non-responsive units
    |> inner_join(units |> rename_with(function(s) str_c(s, "_1")), by="unit_1")
    |> inner_join(units |> rename_with(function(s) str_c(s, "_2")), by="unit_2")
    # exclude pairs on the same electrode (might change this if we calculate distance)
    |> filter(channel_1 != channel_2)
    |> mutate(site=str_match(unit_1, "[:alnum:]+_\\d+_\\d+")[,1])
    |> inner_join(all_sites, by="site")
    |> mutate(conn_type=ifelse(spike_1=="wide", ifelse(spike_2=="wide", "BS-BS", "BS-NS"), ifelse(spike_2=="wide", "BS-NS", "NS-NS")))
)

In [ ]:
## number of pairs by area, condition, and cell type
df <- (
    ucorr
    |> filter(conn_type != "BS-NS")
    |> xtabs(~ area + group + conn_type, data=_)
    |> addmargins(c(1,3))
    |> as.data.frame()
    |> arrange(area)
    |> pivot_wider(names_from=c(area, group), values_from=Freq, values_fill=0)
)
df

In [ ]:
print(xtable(df, digits=0), type="latex")

### Raw data plots

In [ ]:
options(repr.plot.width=4, repr.plot.height=2.5, repr.plot.res = 300)
(
    ucorr
    |> ggplot(aes(conn_type, evoked_noise_c, color=group))
    + facet_grid(~ area)
    + geom_violin()
    + stat_summary(fun.data="mean_se", fatten=1.5, position=position_dodge(width=1.0))
    + theme_classic() + my.theme
)

In [ ]:
options(repr.plot.width=4, repr.plot.height=2.5, repr.plot.res = 300)
(
    ucorr
    |> ggplot(aes(conn_type, spont_noise_c, color=group))
    + facet_grid(~ area)
    + geom_violin()
    + stat_summary(fun.data="mean_se", fatten=1.5, position=position_dodge(width=1.0))
    + theme_classic() + my.theme
)

In [ ]:
options(repr.plot.width=4, repr.plot.height=2.5, repr.plot.res = 300)
(
    ucorr
    |> ggplot(aes(conn_type, signal, color=group))
    + facet_grid(~ area)
    + geom_violin()
    + stat_summary(fun.data="mean_se", fatten=1.5, position=position_dodge(width=1.0))
    + theme_classic() + my.theme
)

### Evoked noise correlation - linear model - pairs

In [ ]:
fm_noise_corr <- (
    ucorr
    |> filter(conn_type != "BS-NS")
    |> lm(evoked_noise_c ~ area*conn_type*group, data=_)
)
emmeans(fm_noise_corr, ~ group | area*conn_type) |> contrast("pairwise")

In [ ]:
joint_tests(fm_noise_corr)

In [ ]:
options(repr.plot.width=1.8, repr.plot.height=1.45, repr.plot.res = 450)
p <- (
    fm_noise_corr
    |> emmeans(~ group*conn_type*area)
    |> confint(level=0.90, type="response")
    |> ggplot(aes(area, emmean, color=group))
    + facet_wrap(~ conn_type)
    + geom_point(position=position_dodge(width=0.5), size=1.5)
    + geom_linerange(aes(ymin=lower.CL, ymax=upper.CL), position=position_dodge(width=0.5))
    + scale_x_discrete(name=NULL)
    + scale_y_continuous("Noise correlation")
    + theme_classic() + my.theme + no.legend
)
p 

In [ ]:
pdf("../figures/noise_correlations.pdf", width=2.4, height=1.75)
print(p)
dev.off()

### Spontaneous noise correlation - linear model - pairs

In [ ]:
fm_corr <- lm(spont_noise_c ~ area*conn_type*group, ucorr)
emmeans(fm_corr, ~ group | area*conn_type) |> contrast("pairwise")

In [ ]:
p <- (
    fm_corr
    |> emmeans(~ group*conn_type*area)
    |> confint(level=0.90, type="response")
    |> ggplot(aes(area, emmean, color=group))
    + facet_wrap(~ conn_type)
    + geom_point(position=position_dodge(width=0.5), size=1)
    + geom_linerange(aes(ymin=lower.CL, ymax=upper.CL), position=position_dodge(width=0.5))
    + scale_x_discrete(name=NULL)
    + theme_classic() + my.theme + no.legend
)
p 

### With random effects

In [ ]:
fm_corr <- lmer(evoked_noise_c ~ area*conn_type*group + (1+conn_type|site), ucorr, control=lmerControl(optimizer="bobyqa"))
emmeans(fm_corr, ~ group | conn_type*area) |> contrast("pairwise")

In [ ]:
p <- (
    fm_corr
    |> emmeans(~ group*conn_type*area)
    |> confint(level=0.90, type="response")
    |> ggplot(aes(conn_type, emmean, color=group))
    + facet_wrap(~ area)
    + geom_pointrange(aes(ymin=asymp.LCL, ymax=asymp.UCL), fatten=1.5, position=position_dodge(width=0.5))
    + theme_classic() + my.theme
)
p 

### Signal correlation - linear model - pairs

In [ ]:
fm_signal_corr <- (
    ucorr
    |> filter(conn_type != "BS-NS")
    |> lm(signal ~ area*conn_type*group, data=_)
)
emmeans(fm_signal_corr, ~ group | area*conn_type) |> contrast("pairwise")

In [ ]:
# compare BS to NS
emmeans(fm_signal_corr, ~ conn_type) |> contrast("pairwise")

In [ ]:
options(repr.plot.width=1.8, repr.plot.height=1.45, repr.plot.res = 450)
p <- (
    fm_signal_corr
    |> emmeans(~ group*conn_type*area)
    |> confint(level=0.90, type="response")
    |> filter(conn_type != "BS-NS")
    |> ggplot(aes(area, emmean, color=group))
    + facet_wrap(~ conn_type)
    + geom_point(position=position_dodge(width=0.5), size=1.5)
    + geom_linerange(aes(ymin=lower.CL, ymax=upper.CL), position=position_dodge(width=0.5))
    + scale_x_discrete(name=NULL)
    + scale_y_continuous("Signal correlation")
    + theme_classic() + my.theme + no.legend
)
p 

In [ ]:
pdf("../figures/signal_correlations.pdf", width=2.4, height=1.75)
print(p)
dev.off()

## Site-level statistics

In [ ]:
ucorr_sites <- (
    ucorr
    |> group_by(group, area, site, conn_type)
    |> filter(n() > 5)
    |> summarize(signal_m=mean(signal), signal_sd=sd(signal), evoked_noise_m=mean(evoked_noise_c), evoked_noise_sd=sd(evoked_noise_c))
)
ucorr_sites

In [ ]:
options(repr.plot.width=4, repr.plot.height=3, repr.plot.res = 300)
(
    ucorr_sites
    |> ggplot(aes(evoked_noise_m, evoked_noise_sd, color=group))
    + facet_wrap(~ conn_type)
    + geom_point()
    + theme_classic() + my.theme + no.legend
)

## Noise vs signal correlations

In [ ]:
options(repr.plot.width=2.6, repr.plot.height=1.85, repr.plot.res = 450)
p <- (
    ucorr
    |> filter(conn_type != "BS-NS", area=="L3/NCM")
    # |> filter(conn_type != "BS-NS")
    |> ggplot(aes(signal, evoked_noise_c, color=group, fill=group))
    + facet_grid(area ~ conn_type)
    + geom_point(size=0.07, alpha=0.2, shape=21)
    + stat_smooth(method="lm", linewidth=0.5)
    + scale_x_continuous("Signal correlation")
    + scale_y_continuous("Noise correlation")
    + theme_classic() + my.theme + no.legend
)
p

In [ ]:
options(repr.plot.width=3.0, repr.plot.height=3.5, repr.plot.res = 450)
p <- (
    ucorr
    |> filter(conn_type != "BS-NS")
    |> ggplot(aes(signal, evoked_noise, color=group, fill=group))
    + facet_grid(area ~ conn_type)
    + geom_point(size=0.1, alpha=0.2, shape=21)
    + stat_smooth(method="lm", linewidth=0.5)
    + scale_x_continuous("Signal correlation")
    + scale_y_continuous("Noise correlation")
    + theme_classic() + my.theme + no.legend
)
p

In [ ]:
pdf("../figures/signal_vs_noise_correlations.pdf", width=3.0, height=3.5)
print(p)
dev.off()

In [ ]:
fm_corr_corr <- (
    ucorr
    #|> filter(conn_type != "BS-NS", area=="L3/NCM")
    |> filter(conn_type != "BS-NS")
    |> lm(evoked_noise ~ signal*conn_type*area*group, data=_)
)
em_corr_corr <- emtrends(fm_corr_corr, pairwise ~ group | area*conn_type, var="signal")
summary(em_corr_corr)

In [ ]:
options(repr.plot.width=1.6, repr.plot.height=3.5, repr.plot.res = 450)
p <- (
    summary(em_corr_corr)$emtrends
    |> ggplot(aes(conn_type, signal.trend, color=group))
    + facet_grid(area ~ .)
    + geom_point(position=position_dodge(width=0.35), size=1.5)
    + geom_linerange(aes(ymin=lower.CL, ymax=upper.CL), position=position_dodge(width=0.35))
    + scale_x_discrete(name=NULL)
    + scale_y_continuous("Slope", limits=c(-0.01, 0.6))
    + theme_classic() + my.theme + no.legend
)
p

In [ ]:
pdf("../figures/signal_vs_noise_corr_slope.pdf", width=1.6, height=3.1)
print(p)
dev.off()

In [ ]:
options(repr.plot.width=1.8, repr.plot.height=1.45, repr.plot.res = 450)
(
    emmip(fm_corr_corr, group ~ signal | conn_type, CIs=T, cov.reduce = range)
    + scale_x_continuous("Signal correlation")
    + scale_y_continuous("Noise correlation")
    + theme_classic() + my.theme + no.legend 
)

In [ ]:
options(repr.plot.width=1.8, repr.plot.height=1.8, repr.plot.res = 450)
(
    ucorr
    |> filter(area=="L3/NCM", conn_type!="BS-NS")
    |> mutate(scorr_group=cut(signal, breaks=c(-1, -0.4, 0.4, 1), labels=c("low", "mid", "high")))
    |> ggplot(aes(scorr_group, evoked_noise_c, color=group))
    + facet_grid(~ conn_type)
    + stat_summary(fun.data="mean_se", fatten=0.1, position=position_dodge(width=0.5))
    + theme_classic() + my.theme + no.legend
)

In [ ]:
# spontaneous vs evoked
options(repr.plot.width=4, repr.plot.height=4, repr.plot.res = 300)
(
    ucorr
    |> ggplot(aes(evoked_noise_c, spont_noise_c, color=group))
    + facet_grid(conn_type ~ area)
    + geom_point()
    + stat_smooth(method="lm")
)

In [ ]:
(
    ucorr
    |> mutate(signal_group=cut(signal, c(-2, -0.4, 0.4, 2), labels=c("negative", "mid", "high")))
    |> ggplot(aes(signal_group, noise_corrected, color=group))
    + facet_grid(conn_type ~ area)
    + stat_summary(fun.data="mean_se", fatten=1.5, position=position_dodge(width=0.5))
    + theme_classic() + my.theme
)

In [ ]:
installed.packages() |> as.data.frame() |> filter(Package == "emmeans")